In [1]:
from kiteconnect import KiteConnect

In [2]:
import logging
import requests

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [9]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [10]:
# Get the Request Token from the redirect URL

request_token = "lfHJw5gPU9776dMecEtHXvnhVA7rxUi6"

# Get Access Token

In [11]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = 'vi1wl6l41InTOc8mUiCEqEo9qnln4k2h'
kite.set_access_token(access_token)
print(access_token)

vi1wl6l41InTOc8mUiCEqEo9qnln4k2h


In [12]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BSE', 'BFO', 'CDS', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# Get Instrument Tokens

In [13]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [14]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

# Get Exponential Moving Average

In [15]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

# Stochastic Momentum Indicator

In [16]:
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# Zerodha Testing LIVE Deployment TradeLog

In [17]:
def get_quote(exchange, tradingsymbol):
    for _ in range(5):
        try:
            quote = kite.quote(exchange+":"+tradingsymbol)[exchange+":"+tradingsymbol]
            break
        except requests.exceptions.ReadTimeout as e:
            logging.exception("Timeout Error - while getting ltp: \n %s", e)
            logging.info("Retrying")
            sleep(1)
            continue
    return quote

In [18]:
def create_candle(price, latest_candle, candle_data, all_prices, current_candle_timestamp):
    # Add latest price to latest_candle data
    latest_candle.loc[len(latest_candle)] = price
    
    # get OHLC for latest candle
    price_list = list(latest_candle['price'])
    actual_high = max(price_list)
    actual_low = min(price_list)
    actual_close = price_list[-1]
    actual_open = price_list[0]
    
    if actual_open < actual_close:
        color = 'green'
    else:
        color = 'red'
    
    # Add to candle data
    candle_data.loc[len(candle_data)] = [datetime.strftime(current_candle_timestamp, "%Y-%m-%d %H:%M:%S"), actual_open, actual_high, actual_low, actual_close, color]
    all_prices = pd.concat([all_prices, latest_candle])
    
    # Update variables for next candle
    latest_candle = pd.DataFrame(columns=['price'])
    
    return latest_candle, candle_data, all_prices, color, price_list

In [19]:
def its_transaction_time(candle_data, short_term_emavg_duration, long_term_emavg_duration, current_candle_timestamp, price_log, current_status, current_transaction_type, trade_log, transaction_price):
    emav_df = pd.DataFrame({'date':list(candle_data['Candle']),'price':list(candle_data['Close'])}).set_index('date', drop=True)
    for window in [short_term_emavg_duration, long_term_emavg_duration]:
        emav_df = get_exponential_moving_avg(emav_df, window)
    current_time = datetime.strftime(current_candle_timestamp, "%Y-%m-%d %H:%M:%S")
    current_price = emav_df.loc[current_time, 'price']
    current_short_term_mavg = emav_df.loc[current_time, str(short_term_emavg_duration)+'d_EMAvg']
    current_long_term_mavg = emav_df.loc[current_time, str(long_term_emavg_duration)+'d_EMAvg']
    price_log.loc[len(price_log)]=[current_time, current_price, current_short_term_mavg, current_long_term_mavg]
    
    if current_short_term_mavg>current_long_term_mavg:
        transaction_type='buy'
        if current_status=='closed':
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            current_price = transaction_price
    elif current_short_term_mavg<current_long_term_mavg:
        transaction_type='sell'
        if current_status=='closed':
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='buy':
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
        elif current_status=='open' and current_transaction_type=='sell':
            current_price = transaction_price
    return price_log, trade_log, current_price, current_transaction_type, current_status

In [20]:
def shut_up_shop(latest_candle, candle_data, all_prices, current_candle_timestamp, short_term_emavg_duration, long_term_emavg_duration, price_log, current_status, current_transaction_type, trade_log):
    # get OHLC for latest candle
    price_list = list(latest_candle['price'])
    actual_high = max(price_list)
    actual_low = min(price_list)
    actual_close = price_list[-1]
    actual_open = price_list[0]
    
    if actual_open < actual_close:
        color = 'green'
    else:
        color = 'red'
    
    # Add to candle data
    candle_data.loc[len(candle_data)] = [datetime.strftime(current_candle_timestamp, "%Y-%m-%d %H:%M:%S"), actual_open, actual_high, actual_low, actual_close, color]
    all_prices = pd.concat([all_prices, latest_candle])
    
    # Update variables for next candle
    latest_candle = pd.DataFrame(columns=['price'])
    
    emav_df = pd.DataFrame({'date':list(candle_data['Candle']),'price':list(candle_data['Close'])}).set_index('date', drop=True)
    for window in [short_term_emavg_duration, long_term_emavg_duration]:
        emav_df = get_exponential_moving_avg(emav_df, window)
    current_time = datetime.strftime(current_candle_timestamp, "%Y-%m-%d %H:%M:%S")
    current_price = emav_df.loc[current_time, 'price']
    current_short_term_mavg = emav_df.loc[current_time, str(short_term_emavg_duration)+'d_EMAvg']
    current_long_term_mavg = emav_df.loc[current_time, str(long_term_emavg_duration)+'d_EMAvg']
    price_log.loc[len(price_log)]=[current_time, current_price, current_short_term_mavg, current_long_term_mavg]
    
    if current_status=='open':
        if current_transaction_type=='buy':
            transaction_type='sell'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
        if current_transaction_type=='sell':
            transaction_type='buy'
            trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
    else:
        pass
    return latest_candle, candle_data, all_prices, color, price_log, trade_log, current_price, current_transaction_type, current_status

In [21]:
def calculate_pnl(current_transaction_type, transaction_price, price):
    if current_transaction_type == 'buy':
        pnl = price - transaction_price
    elif current_transaction_type == 'sell':
        pnl = transaction_price - price
    return pnl

In [22]:
def exit_manoeuvre(price, candle_data, current_candle_timestamp, short_term_emavg_duration, long_term_emavg_duration, current_transaction_type, trade_log):
    emav_df = pd.DataFrame({'date':list(candle_data['Candle'])+[datetime.strftime(current_candle_timestamp, "%Y-%m-%d %H:%M:%S")],'price':list(candle_data['Close'])+[price]}).set_index('date', drop=True)
    for window in [short_term_emavg_duration, long_term_emavg_duration]:
        emav_df = get_exponential_moving_avg(emav_df, window)
    current_time = datetime.strftime(current_candle_timestamp, "%Y-%m-%d %H:%M:%S")
    current_price = emav_df.loc[current_time, 'price']
    current_short_term_mavg = emav_df.loc[current_time, str(short_term_emavg_duration)+'d_EMAvg']
    current_long_term_mavg = emav_df.loc[current_time, str(long_term_emavg_duration)+'d_EMAvg']
    if current_transaction_type == 'buy':
        transaction_type = 'sell'
        trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
    elif current_transaction_type == 'sell':
        transaction_type = 'buy'
        trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=current_time, current_price=current_price, current_short_term_mavg=current_short_term_mavg, current_long_term_mavg=current_long_term_mavg, current_transaction_type=transaction_type)
    return trade_log, current_price, current_transaction_type, current_status

In [23]:
def place_trade(trade_log, current_time, current_price, current_short_term_mavg, current_long_term_mavg, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    trade_log.loc[len(trade_log)] = [current_time, current_price, current_short_term_mavg, current_long_term_mavg, current_transaction_type, current_status]
    print(current_time, current_transaction_type, current_price, current_status)
    return trade_log, current_transaction_type, current_status

# Pre-Flight Checks

### WARNING: Don't forget to update the date and time in 'deply_till' variable

In [24]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '3minute'
short_term_emavg_duration = 4
long_term_emavg_duration = 14

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=9, day=1, hour=15 , minute=30 , second=0)

# first transaction time
candle = 3
transaction_buffer = 3
target = 100
profit_buffer = 40
stop_loss = -100

# THE PAYLOAD

In [25]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 860307


In [ ]:
current_candle_timestamp = datetime(2020, 9, 2, 9, 15, 0)
latest_candle = pd.DataFrame(columns=['price'])
candle_data = pd.DataFrame(columns=['Candle','Open','High','Low','Close','Color'])
all_prices = pd.DataFrame(columns=['price'])
price_log = pd.DataFrame(columns=['time','price',str(short_term_emavg_duration)+'d_EMAvg',str(long_term_emavg_duration)+'d_EMAvg'])
trade_log = pd.DataFrame(columns=['time','price',str(short_term_emavg_duration)+'d_EMAvg',str(long_term_emavg_duration)+'d_EMAvg','transaction_type','status'])
current_status = "closed"
pnl = 0

while datetime.now < deploy_till:
    sleep(1)
    quote = get_quote(exchange, tradingsymbol)
    price = quote['last_price']
    timestamp = datetime.now()
    
    if (current_candle_timestamp+timedelta(minutes=candle)-timestamp).seconds > transaction_buffer:
        latest_candle.loc[len(latest_candle)] = price
    else:
        latest_candle, candle_data, all_prices, color, price_list = create_candle(price, latest_candle, candle_data, all_prices, current_candle_timestamp)
        price_log, trade_log, transaction_price, current_transaction_type, current_status = its_transaction_time(candle_data, short_term_emavg_duration, long_term_emavg_duration, current_candle_timestamp, price_log, current_status, current_transaction_type, trade_log, transaction_price)
        pnl = 0
        current_candle_timestamp = current_candle_timestamp+timedelta(minutes=candle)
        sleep(transaction_buffer-1)
    
    # If end of day is reached
    if datetime.strftime(timestamp, "%H:%M")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M"):
        if current_status == 'open':
            latest_candle, candle_data, all_prices, color, price_log, trade_log, transaction_price, current_transaction_type, current_status = shut_up_shop(latest_candle, candle_data, all_prices, current_candle_timestamp, short_term_emavg_duration, long_term_emavg_duration, price_log, current_status, current_transaction_type, trade_log)
            pnl = 0
        break
    
    # PnL calculation
    pnl = calculate_pnl(current_transaction_type, transaction_price, price)
    
    # stoploss implementation
    if pnl <= stoploss:
        trade_log, current_price, current_transaction_type, current_status = exit_manoeuvre(price, candle_data, current_candle_timestamp, short_term_emavg_duration, long_term_emavg_duration, current_transaction_type, trade_log)
        pnl = 0
    
    # trailing stoploss implementation
    if pnl >= target:
        trailing_stoploss = pnl - profit_buffer
        highest_pnl = pnl
        while pnl:
            quote = get_quote(exchange, tradingsymbol)
            price = quote['last_price']
            timestamp = datetime.now()
            
            if (current_candle_timestamp+timedelta(minutes=candle)-timestamp).seconds > transaction_buffer:
                latest_candle.loc[len(latest_candle)] = price
            else:
                latest_candle, candle_data, all_prices, color, price_list = create_candle(price, latest_candle, candle_data, all_prices, current_candle_timestamp)
                current_candle_timestamp = current_candle_timestamp+timedelta(minutes=candle)
                sleep(transaction_buffer-1)
            
            # If end of day is reached
            if datetime.strftime(timestamp, "%H:%M")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M"):
                latest_candle, candle_data, all_prices, color, price_log, trade_log, transaction_price, current_transaction_type, current_status = shut_up_shop(latest_candle, candle_data, all_prices, current_candle_timestamp, short_term_emavg_duration, long_term_emavg_duration, price_log, current_status, current_transaction_type, trade_log)
                trailing_stoploss = 0
                highest_pnl = 0
                pnl = 0
                break
            
            # Calculate PnL
            pnl = calculate_pnl(current_transaction_type, transaction_price, price)
            
            if pnl < trailing_stoploss:
                # exit current position
                trade_log, current_price, current_transaction_type, current_status = exit_manoeuvre(price, candle_data, current_candle_timestamp, short_term_emavg_duration, long_term_emavg_duration, current_transaction_type, trade_log)
                trailing_stoploss = 0
                highest_pnl = 0
                pnl = 0
                break
            
            elif pnl>trailing_stoploss and pnl<highest_pnl:
                pass
            
            elif pnl>highest_pnl:
                # update trailing stoploss and highest pnl
                trailing_stoploss = pnl - profit_buffer
                highest_pnl = pnl

In [21]:
trade_log.to_excel('LIVE Deployment Logs\RUN1A-HD-20200901-3min_candle_prev_candle_close_4vs14_emavg.xlsx')

In [21]:
(datetime(2020, 9, 1, 13, 0) - datetime(2020, 9, 1, 12, 58)).seconds

120